# **Code for WQD7006 Assignment**

**S2034206 Sun Geng**

This code performs basic Covid-19 related data visualization, modeling, and prediction effect evaluation.

DataSource:	[COVID-19 Data Repository](https://github.com/CSSEGISandData/COVID-19) by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University
<br>Objective1: To visualize and predict the number of COVID-19 confirmed cases in the US using a deep learning method, NeuralProphet.
<br>Objective2: To evaluate and analyze the performance of the NeuralProphet model in the prediction of US COVID-19 confirmed cases.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.offline as py
init_notebook_mode(connected=True)
plt.rcParams.update({'font.size': 14})

Import the data (make sure you update this on a daily basis)

In [2]:
confirmed_df = pd.read_csv('../input/covid19-tme-series-data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('../input/covid19-tme-series-data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('../input/covid19-tme-series-data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [3]:
confirmed_df.head()

In [4]:
confirmed_df.describe()

In [5]:
# Drop date columns if they are mostly NaN
na_columns = (confirmed_df.isna().sum() / confirmed_df.shape[0]) > 0.99
na_columns = na_columns[na_columns]

confirmed_df = confirmed_df.drop(na_columns.index, axis=1)
deaths_df = deaths_df.drop(na_columns.index, axis=1)
recoveries_df = recoveries_df.drop(na_columns.index, axis=1)

In [6]:
## Tidy up the data
confirmed_df = confirmed_df.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='confirmed')
deaths_df = deaths_df.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='deaths')
recoveries_df = recoveries_df.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='recoveries')

In [7]:
confirmed_df['date'] = pd.to_datetime(confirmed_df['date'])
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
recoveries_df['date'] = pd.to_datetime(recoveries_df['date'])

In [8]:
full_df = confirmed_df.merge(recoveries_df).merge(deaths_df)
full_df = full_df.rename(columns={'Country/Region': 'Country', 'date': 'Date', 'confirmed': "Confirmed", "recoveries": "Recoveries", "deaths": "Deaths"})
# Check null values
full_df.isnull().sum()

In [9]:
world_df = full_df.groupby(['Date']).agg({'Confirmed': ['sum'], 'Recoveries': ['sum'], 'Deaths': ['sum']}).reset_index()
world_df.columns = world_df.columns.get_level_values(0)

def add_rates(df):
    df['Confirmed Change'] = df['Confirmed'].diff().shift(-1)
 
    df['Mortality Rate'] = df['Deaths'] / df['Confirmed']
    df['Recovery Rate'] = df['Recoveries'] / df['Confirmed']
    df['Growth Rate'] = df['Confirmed Change'] / df['Confirmed']
    df['Growth Rate Change'] = df['Growth Rate'].diff().shift(-1)
    df['Growth Rate Accel'] = df['Growth Rate Change'] / df['Growth Rate']
    return df

world_df = add_rates(world_df)

# Worldwide Cases

In [10]:
def plot_aggregate_metrics(df, fig=None):
    if fig is None:
        fig = go.Figure()
    fig.update_layout(template='plotly_dark')
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Confirmed'],
                             mode='lines+markers',
                             name='Confirmed',
                             line=dict(color='Yellow', width=2)
                            ))
#     fig.add_trace(go.Scatter(x=df['Date'], 
#                              y=df['Deaths'],
#                              mode='lines+markers',
#                              name='Deaths',
#                              line=dict(color='Red', width=2)
#                             ))
#     fig.add_trace(go.Scatter(x=df['Date'], 
#                              y=df['Recoveries'],
#                              mode='lines+markers',
#                              name='Recoveries',
#                              line=dict(color='Green', width=2)
#                             ))
    return fig

In [11]:
plot_aggregate_metrics(world_df).show()

# Worldwide Rates

In [12]:
def plot_diff_metrics(df, fig=None):
    if fig is None:
        fig = go.Figure()

    fig.update_layout(template='plotly_dark')
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Mortality Rate'],
                             mode='lines+markers',
                             name='Mortality rate',
                             line=dict(color='red', width=2)))

    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Recovery Rate'],
                             mode='lines+markers',
                             name='Recovery rate',
                             line=dict(color='Green', width=2)))

    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Growth Rate'],
                             mode='lines+markers',
                             name='Growth rate confirmed',
                             line=dict(color='Yellow', width=2)))
    fig.update_layout(yaxis=dict(tickformat=".2%"))
    
    return fig

In [13]:
plot_diff_metrics(world_df).show()

# Daily Percent Change in Growth Rate

Useful for tracking whether the growth rate is increasing. Any positive percentage indicates exponential growth.

In [14]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')

tmp_df = world_df.copy()
tmp_df = tmp_df[tmp_df['Growth Rate Accel'] < 10]

fig.add_trace(go.Scatter(x=tmp_df['Date'], 
                         y=tmp_df['Growth Rate Accel'],
                         mode='lines+markers',
                         name='Growth Acceleration',
                         line=dict(color='Green', width=3)))
fig.update_layout(yaxis=dict(tickformat=".2%"))

fig.show()

# Confirmed Cases by Country

In [15]:
confirmed_by_country_df = full_df.groupby(['Date', 'Country']).sum().reset_index()

In [16]:
fig = px.line(confirmed_by_country_df, x='Date', y='Confirmed', color='Country', line_group="Country", hover_name="Country")
fig.update_layout(template='plotly_dark')
fig.show()

In [17]:
# Log scale to allow for view
#  (1) of countries other than China, and
#  (2) identifying linear sections, which indicate exponential growth
fig = px.line(confirmed_by_country_df, x='Date', y='Confirmed', color='Country', line_group="Country", hover_name="Country")
fig.update_layout(
    template='plotly_dark',
    yaxis_type="log"
)
fig.show()

***You can select individual traces above by double-clicking on the legend on the right***

# Top 10 countries by confirmed cases

In [18]:
confirmed_by_country_df.groupby('Country').max().sort_values(by='Confirmed', ascending=False)[:10]

In [19]:
k_layout_kwargs = {
    'font': dict(size=12,),
    'legend': dict(x=0, y=-0.7),
}

# Modeling U.S.

In [20]:
us_df = confirmed_by_country_df[confirmed_by_country_df['Country'] == 'US'].copy()
us_df = add_rates(us_df)

In [21]:
us_df

In [22]:
tmp_df = us_df[us_df['Confirmed'] > 100]

plot_aggregate_metrics(tmp_df).show()

In [23]:
plot_diff_metrics(tmp_df).show()

In [29]:
proxy_country = 'Italy'

confirmed_iran_df = confirmed_by_country_df[(confirmed_by_country_df['Country'] == proxy_country) & (confirmed_by_country_df['Confirmed'] >= 100)]

x = (confirmed_iran_df['Date'] - confirmed_iran_df['Date'].min()).dt.days.to_numpy().reshape(-1, 1)
y = np.log(confirmed_iran_df['Confirmed'])
reg = LinearRegression().fit(x, y)
print(f"Model fit score: {reg.score(x, y):.2f}")
print(f"Growth rate: {reg.coef_[0]:.3f}")
iran_growth_rate = reg.coef_[0]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x[:,0],
        y=np.exp(y),
        name=proxy_country
    )
)

xx = np.linspace(0, len(x[:,0]) + 14, 100)  # Forecast 14 days out
yy = reg.predict(xx.reshape(-1,1))

fig.add_trace(
    go.Scatter(
        x=xx,
        y=np.exp(yy),
        name=f'{proxy_country} - Exponential fit'
    )
)

fig.update_layout(
    title=f"Exponential Model of {proxy_country} Confirmed Cases<br>with 14-Day Extrapolation",
    xaxis_title=f"Days since {confirmed_iran_df['Date'].min()}",
    yaxis_title="Number of Confirmed Cases",
    yaxis_type="log",
    **k_layout_kwargs,
)

fig.show()

The Italy growth rate is at 25%. The U.S. growth rate is at 38%

# U.S. Model with Prophet

We allow for a linear model that detects changepoints at Prophet's default significance. In effect, this is a spline of linear models. This is nice because we expect the growth rate to change at some points during the spread.

In [33]:
from fbprophet.plot import plot_plotly
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot

In [34]:
full_pop = 330e6

#floor_model = lambda x: max(x - 1, 0)  # Use the value itself because the function only increases
floor_model = lambda x: round(0.65 * x)
cap_model = lambda x: round(min(full_pop, 1.5 * x + 10000))  # 50% above plus one to ensure floor > cap at 0

# Modeling Iran confirmed cases 
confirmed_training_df = confirmed_by_country_df[(confirmed_by_country_df['Country'] == 'US') & (confirmed_by_country_df['Confirmed'] > 0)]
confirmed_training_df = confirmed_training_df.rename(columns={'Date': 'ds', 'Confirmed': 'y'}).reset_index(drop=True)

confirmed_training_df['floor'] = confirmed_training_df.y.apply(floor_model)
confirmed_training_df['cap'] = confirmed_training_df.y.apply(cap_model)

In [35]:
confirmed_training_df.y = confirmed_training_df.y.apply(np.log10)
confirmed_training_df.floor = confirmed_training_df.floor.apply(np.log10)
confirmed_training_df.cap = confirmed_training_df.cap.apply(np.log10)

In [36]:
# Total confirmed model 
m = Prophet(
    growth='linear',
    #interval_width=0.90,
    changepoint_prior_scale=0.05,
    changepoint_range=0.9,
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    #n_changepoints=2
)
m.fit(confirmed_training_df)
future = m.make_future_dataframe(periods=14)
future['floor'] = confirmed_training_df.floor
future['cap'] = confirmed_training_df.cap
confirmed_forecast = m.predict(future)

In [37]:
confirmed_forecast

In [38]:
future

In [39]:

confirmed_training_df

The y-axis above is the log of the number of cases. So let's rescale that.

In [40]:
for kwargs in [{}, {"yaxis_type": "log"}]:
    fig = plot_plotly(m, confirmed_forecast, plot_cap=False, changepoints=True)
    annotations = []
    annotations.append(dict(
        xref='paper',
        yref='paper',
        x=0.0,
        y=1.15,
        xanchor='left',
        yanchor='bottom',
        text='Predictions for Confirmed cases U.S. Using Prophet',
        font=dict(
            family='Arial',
            size=30,
            color='rgb(37,37,37)'),
        showarrow=False))
    fig.update_layout(
        annotations=annotations,
        **kwargs
    )
    for trace in fig.data:
        trace.y = np.power(trace.y, 10)
    fig.show()

In [41]:
confirmed_us_df = confirmed_by_country_df[(confirmed_by_country_df['Country'] == 'US')] #& (confirmed_by_country_df['Date'] >= (np.datetime64('today') - np.timedelta64(n_days_to_fit,'D')))]
confirmed_us_df_temp = confirmed_us_df[['Date','Confirmed']]
confirmed_us_df_temp = confirmed_us_df_temp.rename(columns={'Date':'ds','Confirmed':'y'})
confirmed_us_df_temp

# U.S. Model with NeuralProphet

In [42]:
%pip install neuralprophet
import pandas as pd
from fbprophet import Prophet
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_squared_error

# plotting
import matplotlib.pyplot as plt

# settings
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

In [43]:
confirmed_training_df

In [44]:

confirmed_us_df = confirmed_by_country_df[(confirmed_by_country_df['Country'] == 'US')] #& (confirmed_by_country_df['Date'] >= (np.datetime64('today') - np.timedelta64(n_days_to_fit,'D')))]
confirmed_us_df_temp = confirmed_us_df[['Date','Confirmed']]
confirmed_us_df_temp = confirmed_us_df_temp.rename(columns={'Date':'ds','Confirmed':'y'}).reset_index().drop(['index'],axis=1)
confirmed_us_df_temp

df = confirmed_us_df_temp
df

In [45]:
df.plot(x='ds', y='y', title='Log daily page views');

In [46]:

# getting the train/test split
test_length = 365
df_train = df.iloc[:-test_length]
df_test = df.iloc[-test_length:]

In [47]:
prophet_model = Prophet()
prophet_model.fit(df_train)
future_df = prophet_model.make_future_dataframe(periods=test_length)
preds_df_1 = prophet_model.predict(future_df)

In [48]:
prophet_model.plot_components(preds_df_1);

In [49]:
nprophet_model = NeuralProphet()
metrics = nprophet_model.fit(df_train, freq="D")
future_df = nprophet_model.make_future_dataframe(df_train, 
                                                 periods = test_length, 
                                                 n_historic_predictions=len(df_train))
preds_df_2 = nprophet_model.predict(future_df)

In [50]:
nprophet_model.plot(preds_df_2);

In [51]:
nprophet_model.plot_components(preds_df_2, residuals=True);

In [52]:
nprophet_model.plot_parameters();

In [53]:
# prepping the DataFrame
df_test['prophet'] = preds_df_1.iloc[-test_length:].loc[:, 'yhat']
df_test['neural_prophet'] = preds_df_2.iloc[-test_length:].loc[:, 'yhat1']
df_test.set_index('ds', inplace=True)

print('MSE comparison ----')
print(f"Prophet:\t{mean_squared_error(df_test['y'], preds_df_1.iloc[-test_length:]['yhat']):.4f}")
print(f"NeuralProphet:\t{mean_squared_error(df_test['y'], preds_df_2.iloc[-test_length:]['yhat1']):.4f}")

df_test.plot(title='Forecast evaluation');

In [55]:
# from sklearn.metrics import r2_score
# # prepping the DataFrame
# df_test['prophet'] = preds_df_1.iloc[-test_length:].loc[:, 'yhat']
# df_test['neural_prophet'] = preds_df_2.iloc[-test_length:].loc[:, 'yhat1']
# df_test.set_index('ds', inplace=True)

# print('MSE comparison ----')
# print(f"Prophet:\t{r2_score(df_test['y'], preds_df_1.iloc[-test_length:]['yhat']):.4f}")
# print(f"NeuralProphet:\t{r2_score(df_test['y'], preds_df_2.iloc[-test_length:]['yhat1']):.4f}")

# df_test.plot(title='Forecast evaluation');